## Декораторы

Декораторы - вторая важная тема которую мы разобрали. Напомню, что декаратором является любая функция, которая берёт на вход другую функцию и возвращает её модификацию. Это определение подразумивает, что даже лямбда-представления всё ещё подходят под него.

Вот хороший пример с https://cscircles.cemc.uwaterloo.ca/visualize# (отличный сервис который позволяет смотреть историю выполнения интерпритатора, крайне рекомендую с ним ознакомиться. Отдельное спасибо @bobreishestvo за наводку)


In [ ]:
def make_bold(fn):
    return lambda : "<b>" + fn() + "</b>"

def make_italic(fn):
    return lambda : "<i>" + fn() + "</i>"

@make_bold
@make_italic
def hello():
  return "hello world"

helloHTML = hello()

In [ ]:
helloHTML

Декораторы - это достаточно сложная тема. Приведённые ниже задания варьируются от простых, которые требуют просто понимания как устроен декоратор - до продвинутых, которые могут оказаться слишком сложны. Выполнение всех не обязательно, ожидаю что вы попробуете выполнить их и получите некоторую практику самостоятельного написания декораторов.

Для удобства откройте себе ноутбук с занятия про декораторы. Каждый декоратор можно сделать полностью сохраняющим аргументы-документацию-имена функций, это не обязательно но рекомендуется.

С имеющимися знаниями реализуйте:
* Декоратор @even, который будет заменять функцию на идентичную, но каждое нечётное по порядку выполнение вместо исполнения не будет делать ничего (например вместо нормальной логики будет возвращать None и завершать работу.)

In [25]:
import functools
def even(f):
    flag = True
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        nonlocal flag
        if flag:
            f(*args, **kwargs)
            flag = not flag
        else:
            flag = not flag
            return None
    return wrapper


@even
def print_hello(x):
    print("hello", x)

In [26]:
print_hello(1)
print_hello(2)
print_hello(3)
print_hello(4)

hello 1
hello 3


* Декоратор @clip, который пробрасывает в функцию все позиционные аргументы, при этом не пробрасывает ключевые (например @clip от print(1, 2, 3, sep="_") напечатает "1 2 3", не применив sep)


In [27]:
import functools
def clip(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        f(*args)
    return wrapper


@clip
def print_clip(x, y, z = 0, s = "~"):
    print(x, y, z, sep = s)

In [28]:
print_clip(1, 2, z = 3, s = "_")       # 1~2~0
print(1, 2, 3, sep = "_")              # 1_2_3

1~2~0
1_2_3


* Декоратор @repeat(x) (нужно реализовать функцию repeat(x), которая возвращает в качестве результата декоратор), которая выполняет декорируемую функцию x раз (например @repeat(5) от print(1, end="") выведет "11111"), и возвращает тюпл из значений-результатов (например @repeat(5) от sum([1, 2, 3]) вернёт (6, 6, 6, 6, 6))

In [29]:
import random
import functools

def repeat(x):
    def decorator(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            new_list = []
            for i in range(x):
                new_list.append(f(*args, **kwargs))
            new_list = tuple(new_list)
            return new_list
        return wrapper
    return decorator


@repeat(50)
def random_sum(n):
    return sum(random.random() for i in range(n))

In [30]:
random_sum(1_000_000) # должны получить tuple содержащий 50 сумм миллиона случайных чисел от 0 до 1

(500594.880270271,
 499804.32897366484,
 500205.1001403711,
 500306.8816599366,
 500295.5133783027,
 500119.11139345606,
 500013.9344499992,
 500047.95504470536,
 500098.13142105704,
 500704.11482600716,
 500089.5097719361,
 500261.40236906114,
 499668.39515040413,
 499571.43493740016,
 500319.93035194656,
 500160.5477695517,
 500246.8592593074,
 500125.5837393614,
 500153.1481423394,
 500209.3216050758,
 499815.3420163786,
 500020.50460766995,
 499965.9660453478,
 500430.0769225765,
 500080.1377944796,
 500087.2753551447,
 499594.98005078896,
 500200.23285383196,
 499151.13758109923,
 500118.9635624661,
 500399.0412364229,
 500202.70732257026,
 499840.87293501676,
 499703.81351850496,
 499781.28949421336,
 500228.11497048335,
 499932.88967838336,
 500018.8000381965,
 499686.8860273878,
 499582.1024063055,
 500141.199168759,
 500239.4837595054,
 499968.65862682444,
 500169.68487370043,
 500207.20281580894,
 499915.54797185614,
 500043.392777415,
 500463.90914866334,
 500088.59316242434

* Декоратор @cash, который по входным данным проверяет, была ли уже выполнена функция от таких аргументов, и если была - возвращает сохранённое значение. (для простоты можете считать, что функция, которую мы декорируем всегда имеет всего один хешируемый аргумент. Если чувствуете в себе силы, можете попробовать реализовать для общего случая)

Например, если мы имеем простую функцию чисел Фибаначчи:

```
@cash
def fib(x):
  if x < 2:
    return 1
  else:
    return fib(x-1) + fib(x-2)

```

Если вызвать fib(4) без декоратора - он сделает множество вызовов,

```
               1) fib(4)
               /        \
         2) fib(3)   +   6) fib(2)
           /    \               \
    3) fib(2) + 5) fib(1)    7) fib(1)
    /
 4) fib(1)

```

 Вместо этого, при первом вызове fib(1) (4 шаг) мы вернули 1 и запомнили что fib(1) = 1, и теперь при вызове этой функции с аргументом 1 возвращаем 1 без её вычисления (например на шаге 5 и 7).

 Аналогично после завершения на третьем шаге fib(2) - мы запоминаем что он равен двойке и на 6 шаге вместо вызова fib(2) и вызова в нём fib(1) мы так же сразу возвращаем 2

 Хранить значения рекомендую в замыкании декоратора

In [31]:
import functools
def cash(f):
    # ваша реализация внутри
    # для хранения результатов можете, например, создать в этом замыкании словарь cash = dict() где ключами будут аргументы - а значениями return-ы
    cash = dict()
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        if args[0] in cash:
            return cash.get(args[0])
        else:
            res_f = f(*args, **kwargs)
            cash[args[0]] = res_f
            return res_f
    return wrapper

@cash
def fib(x):
  print(f"вызвана фунция Фибоначчи f({x})")
  if x < 2:
    return 1
  else:
    return fib(x-1) + fib(x-2)

In [32]:
fib(10) # здесь при первом вызове функции должны вызваться все f(a) для a из промежутка 0-10

вызвана фунция Фибоначчи f(10)
вызвана фунция Фибоначчи f(9)
вызвана фунция Фибоначчи f(8)
вызвана фунция Фибоначчи f(7)
вызвана фунция Фибоначчи f(6)
вызвана фунция Фибоначчи f(5)
вызвана фунция Фибоначчи f(4)
вызвана фунция Фибоначчи f(3)
вызвана фунция Фибоначчи f(2)
вызвана фунция Фибоначчи f(1)
вызвана фунция Фибоначчи f(0)


89

In [19]:
fib(10) # повторный вызов функции не должен выводить ничего в поток вывода а сразу выдать значения

89

Можем посмотреть на то, насколько быстрее работает наша функция по сравнению с ней же но без декоратора

In [20]:
def old_fib(x):
  if x < 2:
    return 1
  else:
    return old_fib(x-1) + old_fib(x-2)

In [21]:
fib(100)

вызвана фунция Фибоначчи f(100)
вызвана фунция Фибоначчи f(99)
вызвана фунция Фибоначчи f(98)
вызвана фунция Фибоначчи f(97)
вызвана фунция Фибоначчи f(96)
вызвана фунция Фибоначчи f(95)
вызвана фунция Фибоначчи f(94)
вызвана фунция Фибоначчи f(93)
вызвана фунция Фибоначчи f(92)
вызвана фунция Фибоначчи f(91)
вызвана фунция Фибоначчи f(90)
вызвана фунция Фибоначчи f(89)
вызвана фунция Фибоначчи f(88)
вызвана фунция Фибоначчи f(87)
вызвана фунция Фибоначчи f(86)
вызвана фунция Фибоначчи f(85)
вызвана фунция Фибоначчи f(84)
вызвана фунция Фибоначчи f(83)
вызвана фунция Фибоначчи f(82)
вызвана фунция Фибоначчи f(81)
вызвана фунция Фибоначчи f(80)
вызвана фунция Фибоначчи f(79)
вызвана фунция Фибоначчи f(78)
вызвана фунция Фибоначчи f(77)
вызвана фунция Фибоначчи f(76)
вызвана фунция Фибоначчи f(75)
вызвана фунция Фибоначчи f(74)
вызвана фунция Фибоначчи f(73)
вызвана фунция Фибоначчи f(72)
вызвана фунция Фибоначчи f(71)
вызвана фунция Фибоначчи f(70)
вызвана фунция Фибоначчи f(69)
вызвана

573147844013817084101

In [22]:
old_fib(100)

KeyboardInterrupt: 